In [3]:
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector

In [6]:
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
detector = FaceMeshDetector(maxFaces=1)

In [3]:
while True:
    ret, img = cap.read()
    img, faces = detector.findFaceMesh(img, draw=False)
    
    if faces:
        face = faces[0]
        
        pointLeft = face[145]
        pointRight = face[374]
        
        cv2.line(img, pointLeft, pointRight, (0, 200, 0), 3)
        
        cv2.circle(img, pointLeft, 5, (255, 0, 255), cv2.FILLED)
        cv2.circle(img, pointRight, 5, (255, 0, 255), cv2.FILLED)
        
        w, _ = detector.findDistance(pointLeft, pointRight)
        
        
        # Finding the Focal Length
        W = 6.3
        d = 50       
        f = (w*d)/W
        
        print(f)
        
        # Finding distance
        d = (W*f)/w
        
        print(d)
        
        cvzone.putTextRect(img, f'Depth: {d}cm', (face[10][0]-75, face[10][1]-50),
                           scale = 2)                
        
                
    cv2.imshow('Image', img)
    cv2.waitKey(1)

NameError: name 'cap' is not defined

In [4]:
import cv2
import time

def test_camera(index):
    cap = cv2.VideoCapture(index, cv2.CAP_DSHOW)  # Usando CAP_DSHOW para compatibilidad
    if not cap.isOpened():
        print(f"No se pudo abrir la cámara en el índice {index}")
        return

    print(f"Mostrando la cámara en el índice {index}...")
    success, img = cap.read()
    if success:
        cv2.imshow(f'Cámara Índice {index}', img)
        cv2.waitKey(3000)  # Muestra la imagen por 3000 milisegundos (3 segundos)
    else:
        print(f"No se pudo capturar imagen en la cámara índice {index}")

    cap.release()
    cv2.destroyAllWindows()

# Prueba cada cámara disponible
for index in [0, 1]:
    test_camera(index)
    time.sleep(1)  # Espera un segundo antes de probar la próxima cámara para evitar conflictos


Mostrando la cámara en el índice 0...
No se pudo abrir la cámara en el índice 1


In [5]:
from flask import Flask
from flask import render_template
from flask import Response
import cv2
import cvzone
from cvzone.FaceMeshModule import FaceMeshDetector

app = Flask(__name__)
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
detector = FaceMeshDetector(maxFaces=1)

def generate():
     while True:
          ret, frame = cap.read()
          if ret:
               frame, faces = detector.findFaceMesh(frame, draw=False)
               if faces:
                    face = faces[0]
                    pointLeft = face[145]
                    pointRight = face[374]

                    # Calcula la distancia entre los puntos y la profundidad
                    w, _ = detector.findDistance(pointLeft, pointRight)
                    W = 6.3
                    f = 570
                    d = (W * f) / w  # Calcula la nueva distancia usando la longitud focal

                    cvzone.putTextRect(frame, f'Distancia: {round(d,1)}cm', (face[10][0]-125, face[10][1]-50),colorT=(255, 255, 255), 
                            colorR=(255, 0, 0), scale = 2)
                    
                    return frame, cvzone.putTextRect(frame, f'Distancia: {round(d,1)}cm', (face[10][0]-125, face[10][1]-50),colorT=(255, 255, 255), 
                            colorR=(255, 0, 0), scale = 2)
               (flag, encodedImage) = cv2.imencode(".jpg", frame)
               if not flag:
                    continue
               yield(b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' +
                    bytearray(encodedImage) + b'\r\n')

@app.route("/")
def index():
     return render_template("index.html")

@app.route("/video_feed")
def video_feed():
     return Response(generate(),
          mimetype = "multipart/x-mixed-replace; boundary=frame")

if __name__ == "__main__":
     app.run(debug=True)

cap.release()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [24/Apr/2024 15:56:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2024 15:56:16] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2024 15:56:16] "GET /favicon.ico HTTP/1.1" 404 -
